In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("./311_small.csv")


C:\Users\HP\AppData\Local\Temp\ipykernel_4680\2999578992.py:1: DtypeWarning: Columns (8,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./311_small.csv")


In [3]:
df.columns[[8,31,32,34,35,36,37]]

Index(['Incident Zip', 'Vehicle Type', 'Taxi Company Borough',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment'],
      dtype='object')

In [4]:
show_progress = lambda: df["Incident Zip"].isna().sum()
place_desc = ['Borough', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type']
show_progress()


np.int64(88509)

In [5]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032,533 WEST 158 STREET,WEST 158 STREET,AMSTERDAM AVENUE,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303,6 MACORMAC PLACE,MACORMAC PLACE,NORTHFIELD AVENUE,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [6]:
bo = df['Borough'] == "Unspesified"
df.loc[bo, 'Borough'] = np.nan

In [7]:
df['Borough'].value_counts()

Borough
BROOKLYN         322885
QUEENS           240326
BRONX            182361
MANHATTAN        181866
STATEN ISLAND     53961
Unspecified       18502
Name: count, dtype: int64

In [8]:
df['Borough'].isna().sum()

np.int64(98)

In [9]:
for col in place_desc:
    print(f'{col}: {df[col].isna().sum()/df.shape[0]}')

Borough: 9.8000098000098e-05
Incident Zip: 0.08850908850908851
Incident Address: 0.14449214449214448
Street Name: 0.14453714453714453
Cross Street 1: 0.4676954676954677
Cross Street 2: 0.4696934696934697
Intersection Street 1: 0.8754218754218754
Intersection Street 2: 0.8755598755598756
Address Type: 0.08371708371708372
City: 0.08911608911608912
Landmark: 0.9890789890789891
Facility Type: 0.7208647208647209


In [10]:
for col in place_desc:
    print(f'{col:<22}: {(df[col].isna().sum()/df.shape[0]):0.2%}')

Borough               : 0.01%
Incident Zip          : 8.85%
Incident Address      : 14.45%
Street Name           : 14.45%
Cross Street 1        : 46.77%
Cross Street 2        : 46.97%
Intersection Street 1 : 87.54%
Intersection Street 2 : 87.56%
Address Type          : 8.37%
City                  : 8.91%
Landmark              : 98.91%
Facility Type         : 72.09%


### 5 masks for each borough


In [11]:
is_brooklyn = lambda: df['Borough'] == 'BROOKLYN'
is_staten = lambda: df["Borough"] == "STATEN ISLAND"
is_bronx = lambda: df["Borough"] == "BRONX"
is_manhattan = lambda: df["Borough"] == "MANHATTAN"
is_queens = lambda: df["Borough"] == "QUEENS"

In [12]:
df.loc[is_brooklyn(), place_desc].sort_values(by='Street Name').head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
396356,BROOKLYN,NaN,481-483 .DEKALB AVE,.DEKALB AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455159,BROOKLYN,NaN,4821 1 AVENUE,1 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551777,BROOKLYN,11232,4000 1 AVENUE,1 AVENUE,40 STREET,41 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct
170575,BROOKLYN,11220,5401 1 AVENUE,1 AVENUE,54 STREET,55 STREET,54 STREET,55 STREET,NaN,BROOKLYN,1 AVENUE,NaN
669247,BROOKLYN,11220.0,5601 1 AVENUE,1 AVENUE,56 STREET,57 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN


In [13]:
def uniform_names(x):
    pt1 = re.compile(r'\ST.*$')
    pt2 = re.compile(r'\ AVE*\.*$')
    pt3 = re.compile(r'\ RD\.$')
    pt4 = re.compile(r'\ PL\.*$')
    pt5 = re.compile(r'\w{3}')
    try:
        if re.search(pt1, x):
            return re.sub(pt1, " STREET", x).upper()
        elif re.search(pt2, x):
            return re.sub(pt2, " AVENUE", x).upper()
        elif re.search(pt3, x):
            return re.sub(pt3, " ROAD", x).upper()
        elif re.search(pt4, x):
            return re.sub(pt4, " PLACE", x).upper()
        elif not re.search(pt5, x):
            return np.nan
    except:
        return x
    return x

In [14]:
street_cols = ["Incident Address", "Street Name", "Cross Street 1","Cross Street 2","Intersection Street 1","Intersection Street 2"]
for col in street_cols:
    df[col] = df[col].apply(lambda x: uniform_names(x))

In [15]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211,201 SO STREET,SO STREET,SO STREET,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032,533 WE STREET,WE STREET,AM STREET,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303,6 MACORMAC PLACE,MACORMAC PLACE,NO STREET,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [16]:
df.dropna(subset=place_desc, how='all', inplace=True)

In [17]:
show_progress()

np.int64(88509)

In [18]:
tmp = df.loc[is_brooklyn(), place_desc].copy()
tmp.shape

(322885, 12)

In [19]:
tmp.head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
2,BROOKLYN,11211,201 SO STREET,SO STREET,SO STREET,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
8,BROOKLYN,NaN,NaN,NaN,NaN,NaN,69 STREET,AVE Y,INTERSECTION,NaN,NaN,NaN
16,BROOKLYN,11225,410 EA STREET,EA STREET,BEDFORD AVENUE,ROGERS AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
24,BROOKLYN,11228,NaN,NaN,NaN,NaN,12 AVENUE,86 STREET,INTERSECTION,BROOKLYN,NaN,NaN


In [20]:
tmp.sort_values(by=['Street Name', 'Cross Street 1']).head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
3842,BROOKLYN,11223,34 STREET,STREET,STREET,WE STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
6357,BROOKLYN,11217,620 STREET,STREET,STREET,6 AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct
8307,BROOKLYN,11223,32 STREET,STREET,STREET,WE STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
8911,BROOKLYN,11213,1230 STREET,STREET,STREET,ROCHE STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
45882,BROOKLYN,11238,952 STREET,STREET,STREET,GRAND AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN


In [21]:
tmp.shape

(322885, 12)

In [22]:
tmp.drop_duplicates(subset=street_cols, keep='first').shape

(129011, 12)

In [23]:
tmp = tmp.drop_duplicates(subset=street_cols, keep='first')

In [24]:
tmp.head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
2,BROOKLYN,11211,201 SO STREET,SO STREET,SO STREET,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
8,BROOKLYN,NaN,NaN,NaN,NaN,NaN,69 STREET,AVE Y,INTERSECTION,NaN,NaN,NaN
16,BROOKLYN,11225,410 EA STREET,EA STREET,BEDFORD AVENUE,ROGERS AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
24,BROOKLYN,11228,NaN,NaN,NaN,NaN,12 AVENUE,86 STREET,INTERSECTION,BROOKLYN,NaN,NaN


In [25]:
tmp = tmp.dropna(subset=['Street Name', 'Cross Street 1'], how='all')

In [26]:
tmp.groupby(['Street Name', 'Cross Street 1'])['Incident Zip'].unique()

Street Name   Cross Street 1   
 STREET        STREET              [11223, 11217, 11213, 11238, 11216, 11226, 112...
              100  STREET                                                  [11209.0]
              23 AVENUE                                             [11223.0, 11223]
              26 AVENUE                                                    [11223.0]
              27 AVENUE                                                    [11223.0]
                                                         ...                        
YORK  STREET  GOLD  STREET                                          [11201, 11201.0]
              JAY  STREET                                                    [11201]
              MELQUAIN J STREET                                            [11201.0]
              NAVY  STREET                                                 [11201.0]
ZEN STREET    MO STREET                                                    [11225.0]
Name: Incident Zip, Length: 14803

In [27]:
tmp.groupby(['Street Name', 'Cross Street 1'])['Incident Zip'].apply(list)

Street Name   Cross Street 1   
 STREET        STREET              [11223, 11217, 11223, 11213, 11238, 11216, 112...
              100  STREET                                                  [11209.0]
              23 AVENUE                  [11223.0, 11223, 11223.0, 11223.0, 11223.0]
              26 AVENUE                                                    [11223.0]
              27 AVENUE                                                    [11223.0]
                                                         ...                        
YORK  STREET  GOLD  STREET                   [11201, 11201, 11201.0, 11201, 11201.0]
              JAY  STREET                                                    [11201]
              MELQUAIN J STREET                                            [11201.0]
              NAVY  STREET                                                 [11201.0]
ZEN STREET    MO STREET                                                    [11225.0]
Name: Incident Zip, Length: 14803

In [28]:
length = (tmp.groupby(['Street Name', 'Cross Street 1'])['Incident Zip']
          .unique()
          .apply(lambda x: len(x))
)

In [29]:
length[length > 2]

Street Name        Cross Street 1 
 STREET             STREET            23
                   6 AVENUE            3
                   7 AVENUE            4
                   85  STREET          3
                   86  STREET          3
                                      ..
WE STREET          WE STREET           5
WEIRFIELD  STREET  IRVING AVENUE       3
WI STREET          EA STREET           3
WILLOUGHBY AVENUE   STREET             4
WOODPOI STREET     JACKSON  STREET     3
Name: Incident Zip, Length: 547, dtype: int64

In [21]:
dontuse = ['Address Type', 'City', 'Landmark', 'Facility Type']
subset = [col for col in place_desc if col not in dontuse]

def find_unique_zips(boro_mask, attr_list):
    tmp = df.loc[boro_mask, place_desc].copy()   # snapshot per borough
    tmp = tmp.drop_duplicates(subset=subset, keep='first')  # one instance per combination
    tmp = tmp.dropna(subset=attr_list, how='any')  # drop any null value
    
    grouped = tmp.groupby(attr_list)['Incident Zip'].unique()  # pd.Series of np.arrays
    grouped = grouped.apply(lambda x: x[~ np.isnan(x)] if np.isnan(x.sum()) else x)  # if it has one np.nan
    flattened = ( grouped.apply(lambda x: x.flatten()[0] if # extract the float value..
                    (isinstance(x, np.ndarray) and (x.size > 0)) 
                    else np.nan).dropna()  # ...and drop any null values remaining
                )
    return flattened

In [22]:
street_attrs = ['Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
 'Intersection Street 1', 'Intersection Street 2']
from itertools import combinations
for pair in list(combinations(street_attrs, 2)):
    old = show_progress()
    attr_list = ['Borough'] + list(pair)
    for boro_mask in [is_brooklyn, is_bronx, is_queens, is_manhattan, is_staten]:
        flat = find_unique_zips(boro_mask(), attr_list)
        flat = flat.reset_index().set_index(attr_list)
        if not flat.dropna().empty:
            df = df.set_index(attr_list)
            df.update(flat)
            df = df.reset_index()
    print(f'{pair}: {old - show_progress()} zips updated')

TypeError: can only concatenate str (not "float") to str